In [ ]:
# ==============================================================================
# БЛОК 1: ИМПОРТ БИБЛИОТЕК И НАСТРОЙКА ВИЗУАЛИЗАЦИИ
# ==============================================================================
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display

# Настройка для корректного отображения кириллицы на графиках
# plt.rcParams['font.family'] = 'DejaVu Sans' # Можно выбрать другой шрифт, если установлен
sns.set_style("whitegrid")
print("Библиотеки импортированы и настройки применены.")

In [ ]:
# ==============================================================================
# БЛОК 2: ОБЪЕДИНЕНИЕ ДАННЫХ О ПУБЛИКАЦИЯХ (dataset1 + dataset2)
# ==============================================================================
print("\n--- Шаг 1: Обработка данных о публикациях ---")
try:
    df1 = pd.read_csv('dataset1.csv')
    df2 = pd.read_csv('dataset2.csv')

    # Стандартизация названий отраслей
    df1['Industry'] = df1['Industry'].replace('Биоинформатика', 'Биофармация')
    df2['Industry'] = df2['Industry'].replace('Биоинформатика', 'Биофармация')

    # Объединение двух датафреймов
    publications_df = pd.concat([df1, df2], ignore_index=True)

    # Сохранение объединенного файла
    publications_df.to_csv('publications.csv', index=False, encoding='utf-8-sig')
    publications_df.to_excel('publications.xlsx', index=False)

    print("✅ Файлы dataset1.csv и dataset2.csv успешно объединены в publications.csv и publications.xlsx.")
    print(f"Итоговое количество публикаций: {len(publications_df)}")
    display(publications_df.head())

except FileNotFoundError as e:
    print(f"❌ Ошибка: Не найден файл {e.filename}. Убедитесь, что все исходные файлы находятся в нужной директории.")

In [ ]:
# ==============================================================================
# БЛОК 3: ОБРАБОТКА ДАННЫХ О МЕТРИКАХ (dataset3)
# ==============================================================================
print("\n--- Шаг 2: Обработка данных о метриках ---")
try:
    metrics_df = pd.read_csv('dataset3.csv')

    # Стандартизация названий отраслей
    metrics_df['Industry'] = metrics_df['Industry'].replace('Биоинформатика', 'Биофармация')

    # Сохранение очищенного файла
    metrics_df.to_csv('metrics.csv', index=False, encoding='utf-8-sig')
    metrics_df.to_excel('metrics.xlsx', index=False)

    print("✅ Файл dataset3.csv очищен и сохранен как metrics.csv и metrics.xlsx.")
    print(f"Итоговое количество записей о метриках: {len(metrics_df)}")
    display(metrics_df.head())

except FileNotFoundError as e:
    print(f"❌ Ошибка: Не найден файл {e.filename}.")


In [ ]:
# ==============================================================================
# БЛОК 4: АНАЛИЗ И ВИЗУАЛИЗАЦИЯ ДАННЫХ О ПУБЛИКАЦИЯХ
# ==============================================================================
if 'publications_df' in locals():
    print("\n--- Анализ публикаций ---")

    # 1. Аналитическая таблица: Распределение по отраслям и источникам
    publication_summary = publications_df.groupby(['Industry', 'Source']).size().unstack(fill_value=0)
    publication_summary['Total'] = publication_summary.sum(axis=1)

    print("\nТаблица 1: Количество публикаций по отраслям и источникам")
    display(publication_summary)

    # 2. Графики
    plt.figure(figsize=(18, 6))

    # График 1: Общее количество публикаций по отраслям
    plt.subplot(1, 3, 1)
    sns.countplot(y='Industry', data=publications_df, order=publications_df['Industry'].value_counts().index, palette='viridis')
    plt.title('Количество публикаций по отраслям')
    plt.xlabel('Количество')
    plt.ylabel('Отрасль')

    # График 2: Распределение по источникам
    plt.subplot(1, 3, 2)
    source_counts = publications_df['Source'].value_counts()
    plt.pie(source_counts, labels=source_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
    plt.title('Распределение публикаций по источникам')

    # График 3: Распределение отраслей в каждом источнике
    plt.subplot(1, 3, 3)
    sns.countplot(x='Source', hue='Industry', data=publications_df, palette='magma')
    plt.title('Распределение отраслей по источникам')
    plt.xlabel('Источник')
    plt.ylabel('Количество')
    plt.legend(title='Отрасль')

    plt.tight_layout()
    plt.show()

In [ ]:
# ==============================================================================
# БЛОК 5: АНАЛИЗ И ВИЗУАЛИЗАЦИЯ ДАННЫХ О МЕТРИКАХ
# ==============================================================================
if 'metrics_df' in locals():
    print("\n--- Анализ метрик ---")

    # 1. Топ-15 самых частых метрик в целом
    top_metrics_overall = metrics_df['Metric'].value_counts().nlargest(15)

    print("\nТаблица 2: Топ-15 самых частых метрик (в целом по всем отраслям)")
    display(top_metrics_overall.to_frame())

    plt.figure(figsize=(10, 8))
    sns.barplot(x=top_metrics_overall.values, y=top_metrics_overall.index, palette='rocket')
    plt.title('Топ-15 самых частых метрик (в целом)')
    plt.xlabel('Частота упоминания')
    plt.ylabel('Метрика')
    plt.show()

    # 2. Топ-5 метрик для каждой отрасли
    print("\nТаблица 3: Топ-5 метрик для каждой отрасли")

    industries = ['Информатика', 'Экономика', 'Биофармация']

    # Создаем фигуру с 3 подграфиками
    fig, axes = plt.subplots(1, 3, figsize=(20, 8), sharex=True)
    fig.suptitle('Топ-5 метрик по отраслям', fontsize=16)

    for i, industry in enumerate(industries):
        print(f"\n--- {industry} ---")
        top_metrics_industry = metrics_df[metrics_df['Industry'] == industry]['Metric'].value_counts().nlargest(5)
        display(top_metrics_industry.to_frame())

        # Визуализация
        sns.barplot(ax=axes[i], x=top_metrics_industry.values, y=top_metrics_industry.index, palette='coolwarm')
        axes[i].set_title(industry)
        axes[i].set_xlabel('Частота')
        axes[i].set_ylabel('')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()